# Utils

In [1]:
import os
os.chdir("/home/sungwoopark/cs234")

Mounted at /content/drive
/content/drive/MyDrive/GSDS/OhLab/Study/CS234/HW2-coding


In [2]:
!ls

collect_submission.sh  cs234-torch-windows.yml	q5_linear_torch.py
configs		       environment.csv		q6_nature_torch.py
core		       environment.gsheet	q7_train_atari_linear.py
cs234-torch-10.1.yml   Makefile			README.md
cs234-torch-11.0.yml   Problem3.ipynb		utils
cs234-torch-mac.yml    q4_schedule.py


# Solution

In [3]:
import numpy as np
import pandas as pd
import sys
import time
from IPython.display import display

pd.set_option("display.max_columns", None)
sys.setrecursionlimit(10000)

In [4]:
env = pd.read_csv("./environment.csv", engine="python")

In [5]:
env.head()

,state,action,next_state,reward
0,0,0,0,0.1
1,0,1,1,-0.3
2,0,2,2,0.0
3,0,3,3,-0.2
4,0,4,0,0.1


In [6]:
## define environment
states = [i for i in range(4)]
actions = [i for i in range(5)]
TERMINAL = 5
INITIAL_STATE = 0

transition = dict()
for s in states:
    subdf = env[env['state'] == s].reset_index(drop=True)
    subdict = dict()
    for i in range(subdf.shape[0]):
        action, next_state, reward = subdf['action'].iloc[i], subdf['next_state'].iloc[i], subdf['reward'].iloc[i]
        subdict[action] = (next_state, reward)
    transition[s] = subdict

print(transition)

{0: {0: (0, 0.1), 1: (1, -0.3), 2: (2, 0.0), 3: (3, -0.2), 4: (0, 0.1)}, 1: {0: (0, 0.1), 1: (1, -0.3), 2: (2, 0.0), 3: (3, -0.2), 4: (1, -0.3)}, 2: {0: (0, -1.0), 1: (1, 3.0), 2: (2, 0.0), 3: (3, 2.0), 4: (2, 0.0)}, 3: {0: (0, 0.1), 1: (1, -0.3), 2: (2, 0.0), 3: (3, -0.2), 4: (3, -0.2)}}


In [7]:
df = env.copy()
newcol = []
for step in range(TERMINAL):
    newcol += [col+"_"+str(step) for col in env.columns.values]
    df.columns = newcol
    df = pd.merge(left=df, right=env.copy(), left_on=f"next_state_{step}", right_on="state", how="left")

In [8]:
df.shape

(62500, 24)

In [9]:
df = df[df['state_0'] == INITIAL_STATE]
print(df.shape)
display(df.head(10))

(15625, 24)


,state_0,action_0,next_state_0,reward_0,state_1,action_1,next_state_1,reward_1,state_2,action_2,next_state_2,reward_2,state_3,action_3,next_state_3,reward_3,state_4,action_4,next_state_4,reward_4,state,action,next_state,reward
0,0,0,0,0.1,0,0,0,0.1,0,0,0,0.1,0,0,0,0.1,0,0,0,0.1,0,0,0,0.1
1,0,0,0,0.1,0,0,0,0.1,0,0,0,0.1,0,0,0,0.1,0,0,0,0.1,0,1,1,-0.3
2,0,0,0,0.1,0,0,0,0.1,0,0,0,0.1,0,0,0,0.1,0,0,0,0.1,0,2,2,0.0
3,0,0,0,0.1,0,0,0,0.1,0,0,0,0.1,0,0,0,0.1,0,0,0,0.1,0,3,3,-0.2
4,0,0,0,0.1,0,0,0,0.1,0,0,0,0.1,0,0,0,0.1,0,0,0,0.1,0,4,0,0.1
5,0,0,0,0.1,0,0,0,0.1,0,0,0,0.1,0,0,0,0.1,0,1,1,-0.3,1,0,0,0.1
6,0,0,0,0.1,0,0,0,0.1,0,0,0,0.1,0,0,0,0.1,0,1,1,-0.3,1,1,1,-0.3
7,0,0,0,0.1,0,0,0,0.1,0,0,0,0.1,0,0,0,0.1,0,1,1,-0.3,1,2,2,0.0
8,0,0,0,0.1,0,0,0,0.1,0,0,0,0.1,0,0,0,0.1,0,1,1,-0.3,1,3,3,-0.2
9,0,0,0,0.1,0,0,0,0.1,0,0,0,0.1,0,0,0,0.1,0,1,1,-0.3,1,4,1,-0.3


In [10]:
trajectory = []
for i in range(df.shape[0]):
    episode = [tuple(df.iloc[i, j:j+4]) for j in range(0, df.shape[1], 4)]
    trajectory.append(episode)

In [11]:
display(len(trajectory))
display(len(trajectory[0]))
print(trajectory[:10])

15625

6

[[(0.0, 0.0, 0.0, 0.1), (0.0, 0.0, 0.0, 0.1), (0.0, 0.0, 0.0, 0.1), (0.0, 0.0, 0.0, 0.1), (0.0, 0.0, 0.0, 0.1), (0.0, 0.0, 0.0, 0.1)], [(0.0, 0.0, 0.0, 0.1), (0.0, 0.0, 0.0, 0.1), (0.0, 0.0, 0.0, 0.1), (0.0, 0.0, 0.0, 0.1), (0.0, 0.0, 0.0, 0.1), (0.0, 1.0, 1.0, -0.3)], [(0.0, 0.0, 0.0, 0.1), (0.0, 0.0, 0.0, 0.1), (0.0, 0.0, 0.0, 0.1), (0.0, 0.0, 0.0, 0.1), (0.0, 0.0, 0.0, 0.1), (0.0, 2.0, 2.0, 0.0)], [(0.0, 0.0, 0.0, 0.1), (0.0, 0.0, 0.0, 0.1), (0.0, 0.0, 0.0, 0.1), (0.0, 0.0, 0.0, 0.1), (0.0, 0.0, 0.0, 0.1), (0.0, 3.0, 3.0, -0.2)], [(0.0, 0.0, 0.0, 0.1), (0.0, 0.0, 0.0, 0.1), (0.0, 0.0, 0.0, 0.1), (0.0, 0.0, 0.0, 0.1), (0.0, 0.0, 0.0, 0.1), (0.0, 4.0, 0.0, 0.1)], [(0.0, 0.0, 0.0, 0.1), (0.0, 0.0, 0.0, 0.1), (0.0, 0.0, 0.0, 0.1), (0.0, 0.0, 0.0, 0.1), (0.0, 1.0, 1.0, -0.3), (1.0, 0.0, 0.0, 0.1)], [(0.0, 0.0, 0.0, 0.1), (0.0, 0.0, 0.0, 0.1), (0.0, 0.0, 0.0, 0.1), (0.0, 0.0, 0.0, 0.1), (0.0, 1.0, 1.0, -0.3), (1.0, 1.0, 1.0, -0.3)], [(0.0, 0.0, 0.0, 0.1), (0.0, 0.0, 0.0, 0.1), (0.0, 0.0, 

In [12]:
paths = pd.DataFrame(columns=[f"state{i+1}" for i in range(6)]+["reward"])
reward_sum = np.zeros(shape=len(trajectory))
for i, episode in enumerate(trajectory):
    path_dict = dict()
    ep_reward = sum([reward for (_, _, _, reward) in episode[1:-1]]) # do not count the rewards of initial and terminal state
    print(f"Episode {i+1}", end=' ')
    print(f"( ", end='')
    for j, (state, _, _, _) in enumerate(episode):
        path_dict[f"state{j+1}"] = state
        if j < len(episode)-1:
            print(f"{state} -> ", end='')
        else:
            print(f"{state}", end='')
    print(f" )", end=' ')
    print(f"-> Reward {ep_reward:.2f}")
    reward_sum[i] = ep_reward
    path_dict["reward"] = ep_reward
    paths = paths.append(path_dict, ignore_index=True)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Episode 10626 ( 0.0 -> 3.0 -> 2.0 -> 0.0 -> 0.0 -> 0.0 ) -> Reward -0.80
Episode 10627 ( 0.0 -> 3.0 -> 2.0 -> 0.0 -> 0.0 -> 0.0 ) -> Reward -0.80
Episode 10628 ( 0.0 -> 3.0 -> 2.0 -> 0.0 -> 0.0 -> 0.0 ) -> Reward -0.80
Episode 10629 ( 0.0 -> 3.0 -> 2.0 -> 0.0 -> 0.0 -> 0.0 ) -> Reward -0.80
Episode 10630 ( 0.0 -> 3.0 -> 2.0 -> 0.0 -> 0.0 -> 0.0 ) -> Reward -0.80
Episode 10631 ( 0.0 -> 3.0 -> 2.0 -> 0.0 -> 0.0 -> 1.0 ) -> Reward -1.20
Episode 10632 ( 0.0 -> 3.0 -> 2.0 -> 0.0 -> 0.0 -> 1.0 ) -> Reward -1.20
Episode 10633 ( 0.0 -> 3.0 -> 2.0 -> 0.0 -> 0.0 -> 1.0 ) -> Reward -1.20
Episode 10634 ( 0.0 -> 3.0 -> 2.0 -> 0.0 -> 0.0 -> 1.0 ) -> Reward -1.20
Episode 10635 ( 0.0 -> 3.0 -> 2.0 -> 0.0 -> 0.0 -> 1.0 ) -> Reward -1.20
Episode 10636 ( 0.0 -> 3.0 -> 2.0 -> 0.0 -> 0.0 -> 2.0 ) -> Reward -0.90
Episode 10637 ( 0.0 -> 3.0 -> 2.0 -> 0.0 -> 0.0 -> 2.0 ) -> Reward -0.90
Episode 10638 ( 0.0 -> 3.0 -> 2.0 -> 0.0 -> 0.0 -> 2.0 ) -> Reward -0.90
Episode 10639 (

In [13]:
paths.head()

,state1,state2,state3,state4,state5,state6,reward
0,0.0,0.0,0.0,0.0,0.0,0.0,0.4
1,0.0,0.0,0.0,0.0,0.0,0.0,0.4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.4
3,0.0,0.0,0.0,0.0,0.0,0.0,0.4
4,0.0,0.0,0.0,0.0,0.0,0.0,0.4


In [14]:
max_reward = reward_sum.max()
argmaxes = np.where(reward_sum == max_reward)[0]

print(f"Maximum reward {max_reward:.2f} is achieved when an episode is:")

for idx in argmaxes:
    print(f"\t( ", end='')
    for i, (state, _, _, _) in enumerate(trajectory[idx]):
        if i < len(trajectory[idx])-1:
            print(f"{state} -> ", end='')
        else:
            print(f"{state}", end='')
    print(f" )")

Maximum reward 6.10 is achieved when an episode is:
	( 0.0 -> 2.0 -> 1.0 -> 0.0 -> 2.0 -> 1.0 )
	( 0.0 -> 2.0 -> 1.0 -> 0.0 -> 2.0 -> 1.0 )
	( 0.0 -> 2.0 -> 1.0 -> 0.0 -> 2.0 -> 1.0 )
	( 0.0 -> 2.0 -> 1.0 -> 0.0 -> 2.0 -> 1.0 )
	( 0.0 -> 2.0 -> 1.0 -> 0.0 -> 2.0 -> 1.0 )
	( 0.0 -> 2.0 -> 1.0 -> 2.0 -> 1.0 -> 0.0 )
	( 0.0 -> 2.0 -> 1.0 -> 2.0 -> 1.0 -> 0.0 )
	( 0.0 -> 2.0 -> 1.0 -> 2.0 -> 1.0 -> 0.0 )
	( 0.0 -> 2.0 -> 1.0 -> 2.0 -> 1.0 -> 0.0 )
	( 0.0 -> 2.0 -> 1.0 -> 2.0 -> 1.0 -> 0.0 )


In [15]:
paths[(paths['state1'] == 0) & (paths['state2'] == 0) & (paths['state3'] == 2) & (paths['state4'] == 1) & (paths['state5'] == 2) & (paths['state6'] == 1)]

,state1,state2,state3,state4,state5,state6,reward
1430,0.0,0.0,2.0,1.0,2.0,1.0,6.0
1431,0.0,0.0,2.0,1.0,2.0,1.0,6.0
1432,0.0,0.0,2.0,1.0,2.0,1.0,6.0
1433,0.0,0.0,2.0,1.0,2.0,1.0,6.0
1434,0.0,0.0,2.0,1.0,2.0,1.0,6.0
13930,0.0,0.0,2.0,1.0,2.0,1.0,6.0
13931,0.0,0.0,2.0,1.0,2.0,1.0,6.0
13932,0.0,0.0,2.0,1.0,2.0,1.0,6.0
13933,0.0,0.0,2.0,1.0,2.0,1.0,6.0
13934,0.0,0.0,2.0,1.0,2.0,1.0,6.0


* Observations
    * 이 문제 역시 r(s,a,s')의 구조
    * Optimal transition: 2 -> 1; 이 transition을 최대한 많이 가져가면 좋음
    * 제약 조건: 0에서 시작해야 한다
    
    - (2->1)을 두 번 거치는 최단경로: 2 -> 1 -> 2 -> 1 ⇒ 3 actions
    - 이외에도 2 -> 1 -> X -> 2 -> 1 ⇒ 4 actions
    - 0에서 출발하는 1개의 action이 더 추가되므로 2 -> 1의 move를 가져가기 전에 최대 1개의 state만 거쳐갈 수 있음

* 거쳐가는 state X=1, 즉 자기 자신으로 되돌아 간다고 하자.
    - 그렇다면 우선 -0.3의 reward를 얻고, 1 -> 2로 가는 과정에서 0을 얻으므로 총 -0.3의 negative reward가 추가된다. 따라서 not optimal
* X = 3이라고 하면
    - -0.2의 reward와 3 -> 2로 가는 과정에서 0 \rightarrow 총 -0.2의 negative reward
* 따라서 X는 2또는 0이어야 하는데, 
    - X=2라면 2->2로 가는 reward가 0이므로, 마지막에 1 -> 0으로 이동하는 higher value의 경로가 발생한다. 따라서 X=2인 경우는 optimal path의 정의에 위배된다.
    - 따라서 중간에 거쳐가는 state는 반드시 0이어야 하며, 제약조건에 의해서 어딘가를 거쳐가는 경로는 0 -> 2 -> 1 -> 0 -> 2 -> 1 로 고정된다.

* 한편, 0으로부터 출발하면서 2 -> 1 -> 2 -> 1을 거치는 경로는 2가지가 있다.
    - 0 -> 0 -> 2 -> 1 -> 2 -> 1
    - 0 -> 2 -> 1 -> 2 -> 1 -> 0
    - 그런데 initial state에서 전환되는 reward는 count하지 않으므로 첫 번째 경로의 value는 6, 두 번째 경로의 value는 6.1이다.

* 따라서 그 어떤 경로도 6.1보다 높은 reward를 만들 수 없다.
